In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv

In [2]:
# Load the PDF file
loader = PyPDFLoader("G:\VS Code\PDF Reader\SRS Retail Store Surveillance.pdf")
data = loader.load()

In [3]:
# Split data into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300)
docs = text_splitter.split_documents(data)
print("Total number of documents after splitting:", len(docs))

Total number of documents after splitting: 37


In [4]:

# Extract text content
texts = [doc.page_content for doc in docs if isinstance(doc.page_content, str)]

In [5]:
# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



C:\Users\UC\AppData\Local\Temp\ipykernel_10048\2139733256.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
g:\VS Code\PDF Reader\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Create FAISS vectorstore
vectorstore = FAISS.from_texts(texts, embedding_model)
print("FAISS Database is created")

FAISS Database is created


In [7]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})
print('Retriever is set')
print(retriever)

Retriever is set
tags=['FAISS', 'HuggingFaceEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000212C2CC4460> search_kwargs={'k': 1}


In [12]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv

load_dotenv()

llm = HuggingFaceEndpoint(
    #repo_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0", temperature=0.1,
    repo_id="mistralai/Mistral-7B-Instruct-v0.3", temperature=0.1,
    task="text-generation",
    
)

In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    """You are a helpful assistant. Answer all questions in a concise and direct manner.

Guidelines for responses:
- Do NOT include labels like "Human:", "AI:", "Machine:", etc.
- Do NOT reformat answers unless explicitly requested.
- Answer to the point without repeating the question.
- Use proper sentence structure (e.g., "The font size of headings is 14.").
- If asked for a list, provide a clean bullet-point format without unnecessary text.

Avoid:
- Adding unnecessary dialogue formatting.
- Repeating the input question in the response.
- Providing excessive or unrelated details.

Example:
**Question:** What is the font size of subheadings?
**Response:** The font size of subheadings is 13."""
    
  
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

print(prompt)

input_variables=['context', 'input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='You are a helpful assistant. Answer all questions in a concise and direct manner.\n\nGuidelines for responses:\n- Do NOT include labels like "Human:", "AI:", "Machine:", etc.\n- Do NOT reformat answers unless explicitly requested.\n- Answer to the point without repeating the question.\n- Use proper sentence structure (e.g., "The font size of headings is 14.").\n- If asked for a list, provide a clean bullet-point format without unnecessary text.\n\nAvoid:\n- Adding unnecessary dialogue formatting.\n- Repeating the input question in the response.\n- Providing excessive or unrelated details.\n\nExample:\n**Question:** What is the font size of subheadings?\n**Response:** The font size of subheadings is 13.{context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTem

In [18]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [20]:
response = rag_chain.invoke({"input": "what is the purpose of document"})
print(response["answer"])

g:\VS Code\PDF Reader\venv\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


ing functional and nonfunctional requirements?

Response: Documenting functional and nonfunctional requirements is essential for guiding the development process of a software solution. It ensures that the system is built according to the specified requirements, improving its overall quality and usability.
